In [1]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval

import sys

from scripts.robot_arm import RobotArm, RobotArm2D, RobotArm3D, Braccio3D
from scripts.task_info import TaskInfo, numpy_linspace
import scripts.data_prep as data_prep

from scripts.PIBB_helper import qdotdot_gen
from scripts.pdff_kinematic_sim_funcs import get_traj_and_simulate2d, get_traj
import matplotlib.pyplot as plt
from scripts.pdff_kinematic_sim_funcs import PIBB
from scripts.pdff_kinematic_sim_funcs import gen_theta
from scripts.interpolators import IDW, LinearInterpolator
from scripts.pdff_kinematic_sim_funcs import training_data_gen

# Load Data

In [3]:
# data_path = '../training_data/20220327_2215_pibb_2D.csv'
task_info_path = 'training_data/20220327_2215_task_info.csv'
data_path = '/Users/prithvimarwah/Documents/Capstone/intuitive-arm-reach/intuitive-arm-reach/training_data/20220409_1543_pdff_braccio.csv'

pibb_data_df, task_info_df = data_prep.load_data(data_path, task_info_path)

In [4]:
pibb_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   init_joint_angles  657 non-null    object 
 1   x_target           657 non-null    float64
 2   y_target           657 non-null    float64
 3   z_target           657 non-null    float64
 4   Theta              657 non-null    object 
 5   iter_count         657 non-null    int64  
 6   cost               657 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 36.1+ KB


In [5]:
task_info = data_prep.task_info_from_df(task_info_df)

robot_arm = RobotArm2D(
    n_dims = 3,
    link_lengths = np.array([0.6, 0.3, 0.1])
)
task_info.robotarm = robot_arm
task_info = data_prep.clean_task_info(task_info, task_info_df)

In [6]:
main_path = os.getcwd()
# braccio = Braccio()
braccio = Braccio3D()
os.chdir(main_path)
_, task_info = training_data_gen(braccio)


######################### PIBB Algorithm Started #########################
[array([0.        , 0.52359878, 1.57079633, 1.57079633]), array([0, 0, 0, 0])]
######################### PIBB Algorithm Finished. Time Elapsed : 3.752661943435669 #########################


In [7]:
concat_input, flatten_theta, holdout, original_df = data_prep.clean_data(pibb_data_df, task_info, skip_factor=1, planar=False)

Input sizes are: 
 joint angles:  (600, 4) 
x_target:  (600, 1) 
y_target:  (600, 1) 
z_target:  (600, 1)
Output size is:
 theta:  (600, 5, 4)


In [ ]:
np.save("concat_input_all", concat_input)
np.save("flatten_theta", flatten_theta)

In [ ]:
concat_input = np.load("concat_input_all.npy")
flatten_theta = np.load("flatten_theta.npy")


In [ ]:
init_conditions = np.unique(concat_input[:, 0:3], axis=0)
print(len(init_conditions))

for i in range(10):
    filter_condition = init_conditions[i]
    filter = (concat_input[:,0] == filter_condition[0]) & (concat_input[:,1] == filter_condition[1]) & (concat_input[:,2] == filter_condition[2])
    filtered = concat_input[filter]

    c_xs = filtered[:, -2]
    c_ys = filtered[:, -1]
    plt.scatter(x=c_xs, y=c_ys)
    plt.show()

# Training Data Gen

In [ ]:
gen_qs = []

for i, theta in enumerate(flatten_theta):
    reshaped = np.reshape(theta, (task_info.B, task_info.N))
    gen_qdotdot = np.array([qdotdot_gen(task_info, reshaped, t)
                        for t in numpy_linspace(0, task_info.T, task_info.dt)]  )
    init_condit = [np.deg2rad([0, 30, 90, 90]), np.array([0, 0, 0, 0])]
    print(concat_input[i][-3:])
    _, gen_q, _, _ = get_traj(gen_qdotdot, braccio, task_info.dt, init_condit)
    gen_qs.append(gen_q[-1,:])

In [ ]:
pibb_data_df["gen_q"] = gen_qs

In [ ]:
pibb_data_df.to_csv("vroon_is_needy.csv", index=False)

In [ ]:
print(len(gen_qs))

In [ ]:
print(len(concat_input))

In [ ]:
eucs = []
for i in range(len(concat_input)):
    target_x, target_y, target_z = concat_input[i][-3], concat_input[i][-2], concat_input[i][-1]
    reached_x, reached_y, reached_z = braccio.forward_kinematics(gen_qs[i])
    eucs.append(
        np.sqrt(np.abs(reached_z-target_z)**2 + np.abs(reached_y-target_y)**2 + np.abs(reached_x-target_x)**2)
    )
print(np.mean(np.array(eucs)))

In [ ]:
print(np.max(np.array(eucs)))

# Scipy Interpolation

In [8]:
SCALAR = None

In [ ]:
# lin_interp = LinearInterpolator(concat_input, flatten_theta, d=True, scalar=SCALAR)

In [ ]:
# del_interp = LinearInterpolator(concat_input, flatten_theta, d=True, scalar=SCALAR)

In [ ]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler


In [ ]:
scal = MinMaxScaler()

In [ ]:
scaled_angles = scal.fit_transform(concat_input[:, 0:3])

In [ ]:
scaled_angles[0]

In [ ]:
concat_input[0][:3]

In [ ]:
concat_input[:, 0:3]

In [9]:
nn_interp = IDW(concat_input, flatten_theta, K=1, scalar=SCALAR)
idw_interp_3 = IDW(concat_input, flatten_theta, K=3, scalar=SCALAR)
idw_interp_5_s = IDW(concat_input, flatten_theta, K=5, scalar=SCALAR)
idw_interp_5 = IDW(concat_input, flatten_theta, K=5, scalar=None)

In [ ]:
concat_input[0]

In [ ]:
flatten_theta[0]

In [14]:
interp_loaded([0.        , 0.52359878, 1.57079633, 1.57079633, 0.1338345 , 0.        , 0.19113548])

neighbour distances
[4.81447940e-09 5.43375481e-02 5.63443866e-02 6.53902741e-02 6.64917917e-02]
input
[0.0, 0.52359878, 1.57079633, 1.57079633, 0.1338345, 0.0, 0.19113548]
output
[-1.35089663 -1.46529456  1.8553018   8.00738636  0.13290736 -1.19191714  0.49390874  5.46378939 -0.51386026 -0.61098361 -0.65861219  2.97604956 -0.90966718 -0.15420797 -1.0706043   0.52813102  0.91600805 -0.7868862  -0.80293883 -0.88320392]


array([-1.35089663, -1.46529456,  1.8553018 ,  8.00738636,  0.13290736, -1.19191714,  0.49390874,  5.46378939, -0.51386026, -0.61098361, -0.65861219,  2.97604956, -0.90966718, -0.15420797, -1.0706043 ,  0.52813102,  0.91600805, -0.7868862 , -0.80293883, -0.88320392])

In [ ]:
for i in range(5):
    print(idw_interp_5_s.X[:,i].min(), idw_interp_5_s.X[:,i].max())

In [ ]:
idw_interp_5.nearest_neighbour_dist([-1.2203599 ,  2.72228114,  0.07689726,  0.51023788, -0.44950785])


In [ ]:
idw_interp_5.nearest_neighbour_dist([-2.10096494,  1.02539785,  1.82448863, 0.5, 0.5])

# Visualize Results

In [ ]:
def get_preds_df(interpolator):
    preds_dict = {"input":[], "pred_theta":[], "theta":[]}
    for r in range(len(pibb_data_df)):
        if r%1000 == 0:
            print(r)

        pred_t = interpolator(concat_input[r])
        preds_dict["input"].append(concat_input[r])
        preds_dict["pred_theta"].append(pred_t[0])
        preds_dict["theta"].append(flatten_theta[r])
    return pd.DataFrame.from_dict(preds_dict)

In [ ]:
nn_preds = get_preds_df(nn_interp)
idw_preds = get_preds_df(idw_interp)
lin_preds = get_preds_df(lin_interp)

In [ ]:
from IPython import display

def animate_result(init_condition, target_point, robot_arm, interpolator, task_info):

    # print("input: " + str(preds_df["input"][result_index]))
    # print("predicted: " + str(preds_df["pred_theta"][result_index]))
    # print("actual: " + str(preds_df["theta"][result_index]))

    # theta_reshaped = np.reshape(preds_df["pred_theta"][result_index], ( task_info.B, task_info.N))
    # # print(theta_reshaped)

    # test_input = preds_df["input"][result_index]
    # target_pt = [test_input[-2], test_input[-1]]
    # init_condit = [list(test_input[:3]), [0,0,0]]
    input = [*init_condition, *target_point]
    if type(input) == list:
        input = np.array(input)
    theta_pred = interpolator(input)
    theta_reshaped = np.reshape(theta_pred, (task_info.B, task_info.N))
    init_condition = [init_condition, [0,0,0]]

    # print(target_pt)
    # print(init_condit)

    predicted_qdotdot = np.array(  
        [
            qdotdot_gen(task_info, theta_reshaped, t) for t in numpy_linspace(0, task_info.T, task_info.dt)
        ]  
    )
        
    # print(predicted_qdotdot.shape)
    time_steps, q, qdot, gen_qdotdot, ani = get_traj_and_simulate2d(
        qdotdot     = predicted_qdotdot, 
        robot_arm   = robot_arm, 
        x_goal      = target_point, 
        init_condit = init_condition, 
        dt          = task_info.dt
        )

    video = ani.to_jshtml(fps = 60)
    # video = ani.to_html5_video() # to save as mp4, use this
    html = display.HTML(video)
    display.display(html)

In [ ]:
idw_interp_5([-2.10096494,  1.02539785,  1.82448863, 0.5, 0.5])

In [ ]:
concat_input[0]

In [ ]:
test_init_condition = [-1.2203599 ,  2.72228114,  0.07689726]
test_point = [0.5, 0.5]

In [ ]:
animate_result(
    init_condition=test_init_condition,
    target_point=test_point,
    robot_arm=robot_arm,
    interpolator=idw_interp_5,
    task_info=task_info
)

In [ ]:
animate_result(
    init_condition=test_init_condition,
    target_point=test_point,
    robot_arm=robot_arm,
    interpolator=interp_loaded,
    task_info=task_info
)

### Animate Interpolator Trajectory for 5 random points from the dataset

In [ ]:
import random
#Generate 5 random numbers between 10 and 30
randomlist = random.sample(range(0, len(concat_input)), 5)
print(randomlist)

for r in randomlist:
    animate_result(idw_preds, r)

# Unknown Points

In [ ]:
print(len(holdout))

In [ ]:
import math

def gen_trajectory_and_get_euclidean(interpolator, unknown_data, verbose=True):
    pred_unknown = interpolator(unknown_data)
    unknown_reshaped = pred_unknown.reshape(task_info.B, task_info.N)


    predicted_qdotdot = np.array(  
        [
            qdotdot_gen(task_info, unknown_reshaped, t) for t in numpy_linspace(0, task_info.T, task_info.dt)
        ]  
    )
        
    # print(predicted_qdotdot.shape)
    # (qdotdot, robot_arm, x_goal, init_condit, dt)
    time_steps, q, qdot, qdotdot = get_traj(
        predicted_qdotdot, 
        robot_arm, 
        dt = task_info.dt,
        init_condit = [unknown_data[0:3], [0,0,0]]
    )
    
    last_q = q[-1, :]
    reached_pt = robot_arm.forward_kinematics(last_q)
    euc_dist = np.sqrt((reached_pt[0] - unknown_data[-2])**2 + (reached_pt[1] - unknown_data[-1])**2)
    if verbose:
        print("The target pt is: {}; The robot reached {}; Euc distance is : {}".format([unknown_data[-2], unknown_data[-1]], reached_pt, euc_dist))
        print("Theta: " + str(unknown_reshaped))
        print("Data: " + str(unknown_data))
    return euc_dist

In [ ]:
import random

num_configs = 1
num_points = 100

interpolation_styles = ["NN", "3NN_IDW", "5NN_IDW"]
interpolators = [nn_interp, idw_interp_3, idw_interp_5]

totals = [0.0] * len(interpolation_styles)
points = [0] * len(interpolation_styles)
for i in range(5):
    randomlist = random.sample(range(0, len(holdout)), num_configs*num_points)
    for r_id in randomlist:
        init_r = [float(angle) for angle in original_df.iloc[r_id]["init_joint_angles"][1:-1].split()]
        target_r = [original_df.iloc[r_id]["x_target"], original_df.iloc[r_id]["y_target"]]
        unknown_target = [*[*init_r, *target_r]]
        
        for i in range(len(interpolators)):
            euclid = gen_trajectory_and_get_euclidean(interpolators[i], unknown_target, verbose=False)
            if not math.isnan(euclid):
                totals[i] += euclid
                points[i] += 1
                    

avg_euclideans = [0.0] * len(interpolation_styles)
for i in range(len(totals)):
    avg_euclideans[i] = totals[i]/points[i]

print(avg_euclideans)
print(points)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(interpolation_styles, avg_euclideans)
plt.show()


In [ ]:
import random
import math
from scipy.spatial.distance import cosine

def theta_convert(str):
    ret = []
    for r in str[1:-1].split("\n"):
        ts = r.strip()[1:-1].split(" ")
        ts = [t for t in ts if t != ""]
        ts = [float(t) for t in ts]
        ret.extend(ts)
    return ret

num_configs = 1
num_points = 100

interpolation_styles = ["NN", "3NN_IDW", "5NN_IDW"]
interpolators = [nn_interp, idw_interp_3, idw_interp_5]
strat = "SKIPS"

assert(len(interpolation_styles) == len(interpolators))

totals = [0.0] * len(interpolation_styles)
points = [0] * len(interpolation_styles)
for i in range(5):
    randomlist = random.sample(range(0, len(holdout)), num_configs*num_points)
    for r_id in randomlist:
        init_r = [float(angle) for angle in original_df.iloc[r_id]["init_joint_angles"][1:-1].split()]
        target_r = [original_df.iloc[r_id]["x_target"], original_df.iloc[r_id]["y_target"]]
        unknown_target = [*[*init_r, *target_r]]

        for i in range(len(interpolators)):
            pred = interpolators[i](unknown_target)
            if math.isnan(pred[0]):
                continue
            original = theta_convert(original_df.iloc[r_id]["Theta"])
            dist = 1-cosine(pred, original)
            totals[i] += dist
            points[i] += 1
                    

avg_sim = [0.0] * len(interpolation_styles)
for i in range(len(totals)):
    if points[i] == 0:
        avg_sim[i] = 0
    else:
        avg_sim[i] = totals[i]/points[i]

print(avg_sim)
print(points)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(interpolation_styles, avg_sim)
ax.set_title("Avergae Cosine Similarity Between Predicted and Actual Theta")
plt.show()


## Effects of K

In [ ]:
import random 
cosines = []
reach_errors = []

def theta_convert(str):
    ret = []
    for r in str[1:-1].split("\n"):
        ts = r.strip()[1:-1].split(" ")
        ts = [t for t in ts if t != ""]
        ts = [float(t) for t in ts]
        ret.extend(ts)
    return ret

num_k = 30
random_points = random.sample(range(0, len(holdout)), 10)

for i in range(1, num_k):
    interp = IDW(concat_input, flatten_theta, K=i, scalar=SCALAR)
    avg_cosine = []
    avg_reach_error = []
    for r_id in random_points:
        init_r = [float(angle) for angle in original_df.iloc[r_id]["init_joint_angles"][1:-1].split()]
        target_r = [original_df.iloc[r_id]["x_target"], original_df.iloc[r_id]["y_target"]]
        unknown_target = [*[*init_r, *target_r]]
        pred = interp(unknown_target)
        original = theta_convert(original_df.iloc[r_id]["Theta"])
        sim = 1-cosine(pred, original)
        avg_cosine.append(sim)

        # euclid = gen_trajectory_and_get_euclidean(interp, unknown_target, verbose=False)
        # avg_reach_error.append(euclid)

    avg_cosine = np.mean(avg_cosine)
    cosines.append(avg_cosine)

    # avg_reach_error = np.mean(avg_reach_error)
    # reach_errors.append(avg_reach_error)


plt.figure(1)
plt.title("Average Cosine Similarity Between Predicted and Actual Theta\nvs K Nearest Neighbours")
plt.ylabel("Cosine Similarity")
plt.xlabel("K")
plt.plot(range(1,num_k), cosines, linewidth=3)
plt.savefig("cosin_sims.svg", format="svg", dpi=1200)

# plt.figure(2)
# plt.title("Reach Error vs K Nearest Neighbours")
# plt.ylabel("Reach Error")
# plt.xlabel("K")
# plt.plot(range(1,num_k), reach_errors)




In [ ]:
from IPython import display

def animate_traj_to_point(interpolator, input):
    theta_pred = interpolator(input)

    theta_reshaped = np.reshape(theta_pred, ( task_info.B, task_info.N))
    target_pt = [input[-2], input[-1]]
    init_condit = [list(input[:3]), [0,0,0]]

    predicted_qdotdot = np.array(  
        [
            qdotdot_gen(task_info, theta_reshaped, t) for t in numpy_linspace(0, task_info.T, task_info.dt)
        ]  
        )
        
    # print(predicted_qdotdot.shape)
    time_steps, q, qdot, gen_qdotdot, ani = get_traj_and_simulate2d(
        qdotdot     = predicted_qdotdot, 
        robot_arm   = robot_arm, 
        x_goal      = target_pt, 
        init_condit = init_condit, 
        dt          = task_info.dt
        )

    video = ani.to_jshtml(fps = 60)
    # video = ani.to_html5_video() # to save as mp4, use this
    html = display.HTML(video)
    display.display(html)

In [ ]:
x_goal, y_goal = 0.4099025974025974, -0.8449675324675323

In [ ]:
init_condit = concat_input[0][0:3]
input = [*init_condit, x_goal, y_goal]
print(input)
idw_interp_5(input)
# animate_traj_to_point(nn_interp, input)
# animate_traj_to_point(idw_interp_3, input)
# animate_traj_to_point(idw_interp_5, input)

# Save Models

In [11]:
np.save("braccio_small_model_5nn", np.array(idw_interp_5), allow_pickle=True)

In [13]:
interp_loaded = np.load("/Users/prithvimarwah/Documents/Capstone/intuitive-arm-reach/intuitive-arm-reach/src/robot/braccio_small_model_5nn.npy", allow_pickle=True).item()

In [ ]:
interp_loaded = interp_loaded.item()